# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [30]:
import pyorient

In [31]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [32]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [33]:
client.db_create("gods", "graph", "memory")

**Q:** Connect your pyorient client to the `gods` database.

In [34]:
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [35]:
vertex = client.command("CREATE VERTEX CONTENT {name: 'Zeus', symbol: 'thunder'}")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [36]:
client.command("SELECT * FROM V")

**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [37]:
client.command("CREATE VERTEX CONTENT {name:'Héra', symbol:'tiara'}")
client.command("CREATE VERTEX CONTENT {name:'Poséidon', symbol:'trident'}")
client.command("CREATE VERTEX CONTENT {name:'Athena', symbol:'helmet'}")
client.command("CREATE VERTEX CONTENT {name:'Arès', symbol:'weapons'}")

**Q:** Display all vertices with name = `Arès`

In [38]:
for i in client.query("SELECT * FROM V WHERE name = 'Arès'"):
    print(i)

{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [39]:
vertex_zeus = client.command("SELECT * FROM V WHERE name = 'Zeus'")
vertex_poseidon = client.command("SELECT * FROM V WHERE name = 'Poséidon'")

client.command(f"CREATE EDGE E FROM {vertex_zeus[0]._rid} TO {vertex_poseidon[0]._rid} CONTENT {{'kind': 'sibling'}}")

**Q:** Redisplay all vertices, discuss.

In [40]:
for i in client.query("SELECT * FROM V"):
    print(i)

{'@V':{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2128>},'version':2,'rid':'#9:0'}
{'@V':{'name': 'Héra', 'symbol': 'tiara'},'version':1,'rid':'#10:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2358>},'version':2,'rid':'#11:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet'},'version':1,'rid':'#12:0'}
{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Display all edges. They are contained in the class `E`

In [41]:
for i in client.query("SELECT * FROM E"):
    print(i)

{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x00000238811C7C18>, 'in': <pyorient.otypes.OrientRecordLink object at 0x00000238811C7F28>},'version':1,'rid':'#21:0'}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [42]:
client.command(f"CREATE EDGE FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Héra') CONTENT {{'kind': 'sibling'}} ")

client.command(f"CREATE EDGE FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Arès') CONTENT {{'kind': 'father'}} ")

client.command(f"CREATE EDGE FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Athena') CONTENT {{'kind': 'father'}} ")

client.command(f"CREATE EDGE FROM (SELECT FROM V WHERE name = 'Héra') TO (SELECT FROM V WHERE name = 'Arès') CONTENT {{'kind': 'mother'}} ")

client.command(f"CREATE EDGE FROM (SELECT FROM V WHERE name = 'Héra') TO (SELECT FROM V WHERE name = 'Zeus') CONTENT {{'kind': 'sibling'}} ")

client.command(f"CREATE EDGE FROM (SELECT FROM V WHERE name = 'Poséidon') TO (SELECT FROM V WHERE name = 'Zeus') CONTENT {{'kind': 'sibling'}} ")

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [43]:
for i in client.command("SELECT expand(out()) FROM V WHERE name = 'Zeus'"):
    print(i)

{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D28D0>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2860>},'version':3,'rid':'#11:0'}
{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2710>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2080>},'version':4,'rid':'#10:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2400>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2B70>},'version':2,'rid':'#12:0'}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [44]:
for i in client.query("SELECT EXPAND(in) FROM E WHERE kind = 'father'"):
    print(i)

{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811C7CF8>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2860>},'version':2,'rid':'#12:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [45]:
for i  in client.command("SELECT EXPAND(out) FROM E WHERE kind = 'mother'"):
    print(i)

{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000238129A73C8>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2978>},'version':4,'rid':'#10:0'}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [46]:
for i in client.command("SELECT EXPAND(out) FROM E WHERE kind = 'sibling' AND in.name = 'Zeus'"):
    print (i)

{'@V':{'name': 'Héra', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811C7CC0>, 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D26D8>},'version':4,'rid':'#10:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D2048>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000238811D23C8>},'version':3,'rid':'#11:0'}


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [47]:
client.db_create("eCommerce", "graph", "memory")

In [48]:
client.db_open("eCommerce", "root", ROOT_PASSWORD)

In [49]:
client.command("CREATE CLASS Product EXTENDS V")

[11]

In [50]:
client.command("CREATE CLASS Person EXTENDS V")

[12]

In [51]:
client.command("CREATE CLASS Purchased EXTENDS E")

[13]

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [52]:
products = ['spaghetti', 'bolognese sauce', 'cheese', 'apple']

client.command("CREATE VERTEX Product CONTENT {'name':'spaghetti'}")
client.command("CREATE VERTEX Product CONTENT {'name':'bolognese sauce'}")
client.command("CREATE VERTEX Product CONTENT {'name':'cheese'}")
client.command("CREATE VERTEX Product CONTENT {'name':'apple'}")

In [53]:
result = client.query("SELECT * FROM Product WHERE name = 'spaghetti'")

for product in result:
    print(product)

{'@Product':{'name': 'spaghetti'},'version':1,'rid':'#33:0'}


**Q:** Create the following people: `peter`, `meredith`.

In [54]:
peoples = ['peter', 'meredith']

client.command("CREATE VERTEX Person CONTENT {'nickname':'peter'}")
client.command("CREATE VERTEX Person CONTENT {'nickname':'meredith'}")

In [55]:
result2 = client.query("SELECT * FROM Person")

for p in result2:
    print(p)

{'@Person':{'nickname': 'peter'},'version':1,'rid':'#45:0'}
{'@Person':{'nickname': 'meredith'},'version':1,'rid':'#46:0'}


**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [56]:
#peter 20/01/2016
client.command("CREATE EDGE Purchased FROM (SELECT * FROM Person WHERE nickname = 'peter') TO (SELECT * FROM Product WHERE name = 'spaghetti') CONTENT {date:'20/01/2016'}")
client.command("CREATE EDGE Purchased FROM (SELECT * FROM Person WHERE nickname = 'peter') TO (SELECT * FROM Product WHERE name = 'cheese') CONTENT {date:'20/01/2016'}")

#meredith 22/01/2016
client.command("CREATE EDGE Purchased FROM (SELECT * FROM Person WHERE nickname = 'meredith') TO (SELECT * FROM Product WHERE name = 'chesse') CONTENT {date:'22/01/2016'}")
client.command("CREATE EDGE Purchased FROM (SELECT * FROM Person WHERE nickname = 'meredith') TO (SELECT * FROM Product WHERE name = 'apple') CONTENT {date:'22/01/2016'}")
client.command("CREATE EDGE Purchased FROM (SELECT * FROM Person WHERE nickname = 'meredith') TO (SELECT * FROM Product WHERE name = 'bolognese sauce') CONTENT {date:'22/01/2016'}")

#peter 27/01/2016
client.command("CREATE EDGE Purchased FROM (SELECT * FROM Person WHERE nickname = 'peter') TO (SELECT * FROM Product WHERE name = 'spaghetti') CONTENT {date:'27/01/2016'}")
client.command("CREATE EDGE Purchased FROM (SELECT * FROM Person WHERE nickname = 'peter') TO (SELECT * FROM Product WHERE name = 'bolognese sauce') CONTENT {date:'27/01/2016'}")

PyOrientCommandException: com.orientechnologies.orient.core.exception.OCommandExecutionException - No edge has been created because no target vertices: com.orientechnologies.orient.graph.sql.OCommandExecutorSQLCreateEdge$1@1e2e0d4d
	DB name="eCommerce"

**Q:** Who bought Bolognese sauce?

**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.